In [1]:
!pip install langchain langchain-core langchain-community langchain-experimental langchain-openai pyowm google-search-results langchain-groq langsmith wikipedia duckduckgo-search arxiv

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 979.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━

In [157]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')
os.environ["OPENWEATHERMAP_API_KEY"] = userdata.get('OPENWEATHERMAP_API_KEY')

os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_TRACING'] = "true"
os.environ["LANGCHAIN_PROJECT"] = "MultiAgent"

In [158]:
# Build Tools

Duck-Duck-Go

In [159]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

search_wrapper = DuckDuckGoSearchAPIWrapper(max_results=2)
search_tool = DuckDuckGoSearchResults(api_wrapper=search_wrapper, source="news")

In [160]:
search_tool.run("openai")

"snippet: GPT-4o fine-tuning is available today to all developers on all paid usage tiers \u2060 (opens in a new window).. To get started, visit the fine-tuning dashboard \u2060 (opens in a new window), click create, and select gpt-4o-2024-08-06 from the base model drop-down. GPT-4o fine-tuning training costs $25 per million tokens, and inference is $3.75 per million input tokens and $15 per million output ..., title: Fine-tuning now available for GPT-4o - OpenAI, link: https://openai.com/index/gpt-4o-fine-tuning/, snippet: The o1 series excels at accurately generating and debugging complex code. To offer a more efficient solution for developers, we're also releasing OpenAI o1-mini, a faster, cheaper reasoning model that is particularly effective at coding. As a smaller model, o1-mini is 80% cheaper than o1-preview, making it a powerful, cost-effective model for ..., title: Introducing OpenAI o1, link: https://openai.com/index/introducing-openai-o1-preview/, snippet: Prior to GPT-4o, y

In [161]:
from langchain.agents import Tool
from langchain.tools import BaseTool

In [162]:
# defining a single tool
duck_duck_go_tool = Tool(
        name = "search_tool",
        func=search_tool.run,
        description="useful for when you need to answer questions about latest news. You should ask targeted questions"
    )

Wikipedia

In [163]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [164]:
wiki_api_wrapper = WikipediaAPIWrapper(
            top_k_results=1,
            doc_content_chars_max=100
        )
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [165]:
wiki_tool.run('Pythagoras theorme')

'No good Wikipedia Search Result was found'

In [166]:
# defining a single tool
wikipedia_tool = Tool(
        name = "wiki_tool",
        func=wiki_tool.run,
        description="useful when you need to answer questions that references to work in history. You should ask targeted questions"
    )

SerpiAPI

In [167]:
from langchain_community.utilities import SerpAPIWrapper

In [168]:
search_engine = SerpAPIWrapper()

In [169]:
search_engine.run("Coffee shop in michigan?")

'[\'11 Awesome Michigan Coffee Shops, A Day Trip Adventure, Into Detroit Coffee, Coffee at The Gathering Coffee Co., Go On A Grand Rapids Coffee Adventure Trip ...\', \'Explore 74 hand-picked specialty coffee shops and coffee reviews in Michigan, United States on Coffeeopia.\', \'1. Roasting Plant Detroit · 2. The Hot Spot Coffee Company · 3. La Ventana · 4. Shinola Cafe · 5. Percent Coffee · 6. Haraz Coffee House · 7. Lucky Detroit Coffee & ...\', \'We believe that Everyone deserves to feel special. SEE ALL On a mission to bring a name, a face, and a place to every cup of coffee we serve.\', \'Lucky Detroit Coffee & Espresso - Detroit Coffee Shop Located at 2000 Michigan Ave, Detroit 48216.\', \'28 great places to get coffee in Michigan · Astro Coffee · Atomic Coffee · Black Owl Cafe · Chazzano Cafe · Checker Records · Comet Coffee · Common ...\', "There are 5 coffee shops in downtown Holland, Michigan that are sure to satisfy that coffee craving. Let\'s start with the 205 Coffee Bar.

In [170]:
search_engine_tool = Tool(
        name = "search_engine",
        func=search_engine.run,
        description="useful for when you need to answer general user question and need to browse internet. You should ask targeted questions"
    )

In [171]:
# from langchain_groq import ChatGroq

# llm = ChatGroq(
#     model = 'mixtral-8x7b-32768',
#     temperature=0.0
# )

In [172]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.0
)

In [181]:
from langchain.agents import initialize_agent
from langchain.agents import load_tools

my_toolkit = [duck_duck_go_tool, wikipedia_tool, search_engine_tool]
tools = load_tools([], llm=llm)
tools = tools + my_toolkit

In [182]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=2,
    return_messages=True
)

In [183]:
# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [184]:
conversational_agent(" what is the latest news about elections in usa?")



> Entering new AgentExecutor chain...
```json
{
    "action": "search_tool",
    "action_input": "latest news about elections in USA"
}
```
Observation: snippet: Election Live Updates: Harris Calls Trump 'Unfit,' Citing Former Aide's Interview. Vice President Kamala Harris cited remarks by John Kelly, Mr. Trump's former White House chief of staff ..., title: Election Live Updates: Harris Calls Trump 'Unfit,' Citing Former Aide's ..., link: https://www.nytimes.com/live/2024/10/23/us/harris-trump-election/heres-the-latest-on-the-presidential-race, snippet: Election Day is just 14 days out, and Vice President Kamala Harris and former President Donald Trump are hitting the campaign trail to make their final pitches to voters. Follow for live updates., title: Live updates: Trump-Harris election news | CNN Politics, link: https://www.cnn.com/politics/live-news/trump-harris-election-10-22-24/index.html, snippet: Voters in Hawaii, Missouri, Utah, and Wisconsin started casting their ballots o

{'input': ' what is the latest news about elections in usa?',
 'chat_history': [],
 'output': "The latest news about elections in the USA includes Vice President Kamala Harris calling Trump 'unfit' based on remarks by John Kelly, Trump's former White House chief of staff. Election Day is approaching with Harris and Trump campaigning actively. Early voting has started in several states. For more details, you can visit the following links: [NY Times](https://www.nytimes.com/live/2024/10/23/us/harris-trump-election/heres-the-latest-on-the-presidential-race), [CNN Politics](https://www.cnn.com/politics/live-news/trump-harris-election-10-22-24/index.html), [Al Jazeera](https://www.aljazeera.com/news/2024/10/23/us-election-13-days-left-what-polls-say-what-harris-and-trump-are-up), [Reuters](https://www.reuters.com/world/us/us-election-live-harris-trump-locked-race-white-house-2024-10-21/)"}

In [185]:
conversational_agent("Do we have anu NFL game today?")



> Entering new AgentExecutor chain...
```json
{
    "action": "search_tool",
    "action_input": "NFL games schedule today"
}
```
Observation: snippet: The Sporting News has you covered with everything that you need to know to watch the NFL today. NFL games on TV today Thursday, Oct. 17. Game Time (ET) ... Full NFL schedule for all 272 games in 2024;, title: NFL games on TV today: Full schedule, times, channels, live streams to ..., link: https://www.sportingnews.com/us/nfl/news/nfl-games-tv-today-schedule-times-channels-live-streams-watch/23bcab387c84e5144f0d5a04, snippet: LIVE: Follow Bears vs. Jaguars updates from NFL London game. NFL games on TV today Thursday, Oct. 10. Game Time (ET) TV channel; ... Full NFL schedule for all 272 games in 2024;, title: NFL games on TV today: Full schedule, times, channels, live streams to ..., link: https://www.sportingnews.com/us/nfl/news/nfl-games-tv-today-schedule-times-channels-live-streams-watch/949170efa23ed2b71da4b6b8, snippet: Here's what

{'input': 'Do we have anu NFL game today?',
 'chat_history': [HumanMessage(content=' what is the latest news about elections in usa?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The latest news about elections in the USA includes Vice President Kamala Harris calling Trump 'unfit' based on remarks by John Kelly, Trump's former White House chief of staff. Election Day is approaching with Harris and Trump campaigning actively. Early voting has started in several states. For more details, you can visit the following links: [NY Times](https://www.nytimes.com/live/2024/10/23/us/harris-trump-election/heres-the-latest-on-the-presidential-race), [CNN Politics](https://www.cnn.com/politics/live-news/trump-harris-election-10-22-24/index.html), [Al Jazeera](https://www.aljazeera.com/news/2024/10/23/us-election-13-days-left-what-polls-say-what-harris-and-trump-are-up), [Reuters](https://www.reuters.com/world/us/us-election-live-harris-trump-locked-race-white-house-2024-10-21/

In [186]:
conversational_agent("Who is Newton?")



> Entering new AgentExecutor chain...
```json
{
    "action": "search_tool",
    "action_input": "Newton"
}
```
Observation: snippet: Isaac Newton (1642-1726) was a key figure in the Scientific Revolution and the Enlightenment, who developed classical mechanics, universal gravitation, calculus, optics, and alchemy. He is often regarded as the father of physics for his contributions to natural philosophy and astronomy., title: Isaac Newton - Wikipedia, link: https://en.wikipedia.org/wiki/Isaac_Newton, snippet: Isaac Newton (born December 25, 1642 [January 4, 1643, New Style], Woolsthorpe, Lincolnshire, England—died March 20 [March 31], 1727, London) was an English physicist and mathematician who was the culminating figure of the Scientific Revolution of the 17th century. In optics, his discovery of the composition of white light integrated the phenomena of colours into the science of light and ..., title: Isaac Newton | Biography, Facts, Discoveries, Laws, & Inventions ..., link: http

{'input': 'Who is Newton?',
 'chat_history': [HumanMessage(content=' what is the latest news about elections in usa?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The latest news about elections in the USA includes Vice President Kamala Harris calling Trump 'unfit' based on remarks by John Kelly, Trump's former White House chief of staff. Election Day is approaching with Harris and Trump campaigning actively. Early voting has started in several states. For more details, you can visit the following links: [NY Times](https://www.nytimes.com/live/2024/10/23/us/harris-trump-election/heres-the-latest-on-the-presidential-race), [CNN Politics](https://www.cnn.com/politics/live-news/trump-harris-election-10-22-24/index.html), [Al Jazeera](https://www.aljazeera.com/news/2024/10/23/us-election-13-days-left-what-polls-say-what-harris-and-trump-are-up), [Reuters](https://www.reuters.com/world/us/us-election-live-harris-trump-locked-race-white-house-2024-10-21/)", additional_k

In [179]:
conversational_agent("What are some of the coolest place to visit in michigan?")



> Entering new AgentExecutor chain...
```json
{
    "action": "search_engine",
    "action_input": "Coolest places to visit in Michigan"
}
```
Observation: [{'title': 'Detroit Institute of Arts', 'link': 'https://www.google.com#', 'description': 'Art museum', 'rating': 4.8, 'reviews': 12000, 'price': '$23.50', 'extracted_price': 23.5, 'thumbnail': 'https://serpapi.com/searches/67193b132e33c8244ee1cdfe/images/f21325255fc6294e64ba0ee799341b226999376177866d1c16bfdc9166143df3.jpeg'}, {'title': 'Henry Ford Museum of American Innovation', 'link': 'https://www.google.com#', 'description': 'Museum', 'rating': 4.8, 'reviews': 28000, 'price': '$46.00', 'extracted_price': 46.0, 'thumbnail': 'https://serpapi.com/searches/67193b132e33c8244ee1cdfe/images/f21325255fc6294e64ba0ee799341b22dc8d55c6ed2a2d225b5d44de0339daa7.jpeg'}, {'title': 'Frederik Meijer Gardens & Sculpture Park', 'link': 'https://www.google.com#', 'description': 'Botanical garden', 'rating': 4.8, 'reviews': 13000, 'price': '$20.00'

{'input': 'What are some of the coolest place to visit in michigan?',
 'chat_history': [HumanMessage(content='Do we have anu NFL game today?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The NFL games schedule for today includes matchups such as New England Patriots vs. Jacksonville Jaguars and Seattle Seahawks at Atlanta Falcons. You can find more details on the schedule and how to watch the games online or on TV.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who is Newton?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Isaac Newton was an English physicist and mathematician who made groundbreaking contributions to physics, mathematics, astronomy, and more. He is known for his laws of motion, universal gravitation, and calculus. Newton's work has had a profound impact on the field of science and his legacy continues to influence modern physics and mathematics.", additional_kwargs={}, response_metadata={})],
 'output':

In [187]:
result =conversational_agent(" what is the temperature in michigan?")



> Entering new AgentExecutor chain...
```json
{
    "action": "search_engine",
    "action_input": "current temperature in Michigan"
}
```
Observation: {'type': 'weather_result', 'temperature': '67', 'unit': 'Fahrenheit', 'precipitation': '1%', 'humidity': '59%', 'wind': '11 mph', 'location': 'Michigan', 'date': 'Wednesday 2:00 PM', 'weather': 'Mostly cloudy'}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The current temperature in Michigan is 67 degrees Fahrenheit with 1% precipitation, 59% humidity, and 11 mph wind. The weather is mostly cloudy as of Wednesday at 2:00 PM."
}
```

> Finished chain.


In [189]:
result['output']

'The current temperature in Michigan is 67 degrees Fahrenheit with 1% precipitation, 59% humidity, and 11 mph wind. The weather is mostly cloudy as of Wednesday at 2:00 PM.'